In [1]:
import geopandas as gpd
import pandas as pd
import json
import os

# --- Konfigurasi Input dan Output ---
# Ganti dengan path file Anda yang sebenarnya
SHP_FILE = 'PetaLengkap-KomoditasdanPOI-SumateraUtara/PetaLengkap-KomoditasdanPOI-SumateraUtara.shp' 
POI_EXCEL_FILE = 'DataPOI.xlsx' 

# Nama file output untuk frontend
GEOJSON_OUTPUT = 'data_desa.geojson'
POI_JSON_OUTPUT = 'data_poi.json'

print("➡️ Memulai konversi data...")

# --- 1. Konversi SHP ke GeoJSON (Data Desa/Wilayah) ---
try:
    print(f"   Membaca Shapefile: {SHP_FILE}")
    gdf = gpd.read_file(SHP_FILE)
    
    # Cleaning dan Simplifikasi (Mirip dengan kode Streamlit asli)
    gdf['geometry'] = gdf.geometry.simplify(0.001, preserve_topology=True)
    gdf = gdf.rename(columns={'WADMKK': 'Kabupaten'}) 
    gdf['Kabupaten'] = gdf['Kabupaten'].fillna('N/A') 
    
    # Pastikan kolom kunci ada
    required_cols = ['Kabupaten', 'Prediksi', 'jumlah_poi']
    for col in required_cols:
        if col not in gdf.columns:
            print(f"   ⚠️ Peringatan: Kolom '{col}' tidak ditemukan di SHP. Menambahkan kolom kosong.")
            gdf[col] = None
    
    # Dropna untuk memastikan data GeoJSON bersih
    gdf = gdf.dropna(subset=['Kabupaten']) 

    # Konversi ke GeoJSON dan simpan
    gdf_json = gdf.to_json()
    with open(GEOJSON_OUTPUT, 'w') as f:
        f.write(gdf_json)
    
    print(f"   ✅ GeoJSON desa berhasil disimpan di: {GEOJSON_OUTPUT}")

except FileNotFoundError:
    print(f"   ❌ ERROR: File Shapefile tidak ditemukan di path: {SHP_FILE}")
    exit()
except Exception as e:
    print(f"   ❌ ERROR saat memproses SHP: {e}")
    exit()

# --- 2. Konversi Excel POI ke JSON ---
try:
    print(f"   Membaca Excel POI: {POI_EXCEL_FILE}")
    poi_df = pd.read_excel(POI_EXCEL_FILE)
    poi_df = poi_df.dropna(subset=['longitude', 'latitude'])
    
    # Pastikan kolom kunci ada
    if 'category' not in poi_df.columns:
        poi_df['category'] = 'Umum' 
    if 'Kabupaten' not in poi_df.columns:
        poi_df['Kabupaten'] = 'N/A'
    
    # Konversi ke format JSON record dan simpan
    poi_json = poi_df.to_json(orient='records')
    with open(POI_JSON_OUTPUT, 'w') as f:
        f.write(poi_json)
        
    print(f"   ✅ Data POI berhasil disimpan dalam JSON di: {POI_JSON_OUTPUT}")

except FileNotFoundError:
    print(f"   ❌ ERROR: File POI Excel tidak ditemukan di path: {POI_EXCEL_FILE}")
except Exception as e:
    print(f"   ❌ ERROR saat memproses Excel POI: {e}")

print("\n🎉 Pra-proses data selesai. Anda sekarang memiliki file 'data_desa.geojson' dan 'data_poi.json' untuk diakses oleh `script.js`.")

➡️ Memulai konversi data...
   Membaca Shapefile: PetaLengkap-KomoditasdanPOI-SumateraUtara/PetaLengkap-KomoditasdanPOI-SumateraUtara.shp
   ✅ GeoJSON desa berhasil disimpan di: data_desa.geojson
   Membaca Excel POI: DataPOI.xlsx
   ✅ Data POI berhasil disimpan dalam JSON di: data_poi.json

🎉 Pra-proses data selesai. Anda sekarang memiliki file 'data_desa.geojson' dan 'data_poi.json' untuk diakses oleh `script.js`.
